How-To Create a Visual Retention Analysis
======================

An introduction to building and analyzing a user retention analysis with Node.js and MongoDB.

## Introduction

Technology startups, and their evaluations, are at an all time high. Companies in seemingly every industry are emerging with groundbreaking new technologies that allow them to reach and change the way millions of users across the world go about their daily lives.  Uber, for example, is building a logistics system that can dispatch rides and deliveries to their customers in under 5 minutes, across the globe. Gone are the days of hailing a cab in the pouring rain, hoping one will pick you up. Now at the touch of a button you can request a ride, know exactly how much you are going to pay, and even split the bill with someone going in a similar direction. As impressive as the technology is today, it took a lot of research and development to get to this point. Perhaps one of the most important pieces of research in any growing startup is Retention: Are users coming back for more? If so, how often?

Retention is one of the most important pieces of analysis a company can do to determine whether or not their product is successful and sticking with their users. If users aren’t coming back, then it does not matter how amazing the technology stack is, you do not have a viable company. Investors know this better than anyone, and it will be one of the number one things they look at when valuing a business.

In this How-To paper I will look at building a basic retention library in JavaScript that can analyze any two data points in a database. The caller of the library will be able to select a time period for retention (daily, weekly, monthly, etc.) and then provide two functions: one for determining the initial set of users for a given date, and one for determining a retained set of users for a given date. The library will then use those two function to generate a full retention analysis for a given time period. In the results section of the report I will generate retention reports using data from my company, Tablelist. Tablelist provides instant access to high end clubs and lounges in cities across the U.S. I will be analyzing the retention of users who buy different types of items in the iOS application and then return to buy something else.

## Method

A retention report consists of two main pieces: the initial dataset, and the retained dataset based on the initial dataset. Often times, the initial set is a unique list of users bucketed out by some factor. For instance: age, gender, country, etc. Of the unique buckets, you want to learn something about the behavior of those users. For instance: did the user make a purchase 1 month after signing up? 2 months after signing up? 3 months, etc… This is valuable insight into user behavior for your product and can product really powerful results. 

I chose to implement the retention report using Node.js and MongoDB. MongoDB is one of the most popular databases in use today and Node.js has the largest open source repository of software in the world. The library is built like any standard library in the NPM registry, you can import the lib with `require(‘retention’)` and then call the `generate` function by passing in a time period (“day”, “week”, “month”, “year”) and the number of periods you want to generate. The final two parameters are the core of the library and the main method to this project.

The third parameter is a function which is expected to accept a base MongoDB query with constraints on the `created` property. This allows the library to dynamically call this function with different date ranges. The function is expected is to produce a unique set of values for that date range based on whatever retention you would like to run. For example, if I wanted to run retention on all users who made a reservation and then made a second reservation sometime in the future, I would provide the unique list of users ids who made a reservation in the given time period. This is clearly seen in the `book-book.js` example.

The final parameter is similar to first parameter in that it is a function that accepts a base MongoDB query, but it also expects an existing list of unique ids that must be used to perform another lookup taking into account the given list so we know what subset of that list was retained. With these 2 functions, the retention library has enough to work with to create any retention report over any time period. First the library generates an initial set of ids for each time period. For example, if passed “month” and 12 sections, it would calculate an initial set of ids for the each of the past 12 months. The second function is then called for every time period after the initial set, until now. This creates a pyramid like diagram where you can see the retention 1 month out, 2 months out, 3 months out etc. The retention for last one will only have one block since there has only been 1 month since last. 2 months ago will have 2 block, 3 months ago will have 3, etc.

## Results

With a working retention library in place, I wanted to show off some features on production data from Tablelist. Tablelist stores reservations in a MongoDB collection called “Bookings”. The collection contains 1 document per reservation. Each reservation has a reference to a user, “userId”, a status (“processed”, “pending”, “voided”) and some payment information like how much the user spent, tax, tip, etc. Our goal is to find the user retention of people who complete a booking in a given month and then x number of months later. As a business it is important for us to show investors that we expect users to complete a reservation about 4 times a year.

In the “example” folder I have a script called “book-book.js” which requires our retention library and runs retention on a monthly basis for the past 12 months. It passes 2 functions to the library, `initIds` and `retainedIds`. `initIds` provides an initial set of user ids to base the retention off of. The retention library will call this function once for each of the past 12 months. `retainedIds` is similar to `initIds` but it is passed in a list of ids that we need to attempt to retain. In other words, we don’t just want all reservation in a given time period, but we want all reservations in that time period made by anyone in the passed in list.  This will allow us to find the users who booked first and then booked again.

You can see the results of this in the results folder. The `print` function on the retention object prints a CSV representation of the report which is perfect for opening in a variety of applications. I ran two reports, one for 2014 and one for 2015, and manipulated each report in excel in a Google Sheets to show that CSV makes it easy to open between programs. 

## Discussion

Retention analysis provides a great first look at user behavior for any product due to it’s dynamic nature of taking an initial dataset and comparing it to any number of metrics that have a relationship to the initial set. This provides a lot of flexibility to look at any aspect of a product from multiple angles. With that said, retention does not tell the full picture. Where retention falls short in reporting is: revenue and the cost to acquire a certain type of user behavior. As a company it’s important to understand retention but it’s also important to understand how that retention is affected by marketing and operations spend. Retention is a great tool for analyzing a specific marketing campaign over a specific time period, but retention on it’s own doesn’t provide enough insight into a successful business.

## References

#### Intercom.io
[Retention, Cohorts, and Visualisations](https://blog.intercom.io/retention-cohorts-and-visualisations/)

#### Keen.io
[How To Do a Retention Analysis](https://keen.io/blog/47823687779/how-to-do-a-retention-analysis/)

#### Mixpanel
[Retention](https://mixpanel.com/retention/)